In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import safetensors.torch as safe

l = safe.load_file

In [3]:
s6c5 = l(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/gen_act/reasondata-b-2d-l15/blocks.15.hook_resid_post/shard-6-chunk-00000005.safetensors"
)
s6c5["activation"].shape

torch.Size([1, 8192, 4096])

In [4]:
print(s6c5["tokens"][0].tolist())

[128000, 128016, 48, 25, 220, 128017, 10267, 198, 79145, 69, 2120, 11992, 59, 9594, 2120, 61, 18, 12, 19, 87, 61, 17, 10, 20, 87, 12, 17, 73441, 933, 2746, 584, 1095, 400, 69, 48922, 7, 74, 9317, 3, 79164, 279, 400, 74, 3, 339, 32905, 315, 400, 69, 55976, 12849, 400, 69, 48922, 7, 605, 9317, 7, 16, 8, 13244, 1789, 279, 25491, 315, 420, 3575, 11, 5296, 430, 400, 605, 5947, 18509, 19272, 15, 13244, 128018, 32, 25, 220, 128019, 128020, 198, 33413, 11, 1095, 596, 1518, 13, 358, 1205, 311, 12849, 279, 220, 605, 339, 32905, 315, 279, 734, 282, 2120, 8, 284, 8119, 2120, 44301, 482, 220, 19, 87, 30556, 489, 220, 20, 87, 482, 220, 17, 8, 26126, 520, 865, 284, 220, 16, 13, 89290, 11, 430, 10578, 17395, 13, 2030, 7344, 1070, 596, 264, 14397, 1618, 1606, 315, 279, 3230, 1486, 865, 284, 220, 16, 13, 6914, 757, 1781, 382, 5451, 11, 1095, 757, 24564, 279, 5811, 4871, 279, 76359, 734, 25, 865, 44301, 482, 220, 19, 87, 30556, 489, 220, 20, 87, 482, 220, 17, 13, 10926, 422, 358, 20206, 304, 865, 284, 22

In [5]:
import torch


def filter_and_pad_tensor(tokens, activation):
    original_length = len(tokens)
    tokens_list = tokens.tolist()
    activation_list = activation.tolist()
    indices_to_remove = set()

    k = len(tokens) - 1
    if tokens[k] >= 128000:
        pad_token_id = tokens[k]
        while k >= 0 and tokens[k] == pad_token_id:
            k -= 1

    i = 1
    while i <= k:
        if tokens_list[i] >= 128000:
            found = False
            for j in range(i + 1, min(i + 6, len(tokens_list))):
                if tokens_list[j] >= 128000:
                    indices_to_remove.update(range(i, j + 1))
                    i = j
                    found = True
                    break
            if not found:
                indices_to_remove.add(i)
        i += 1

    filtered_tokens_list = [tokens_list[idx] for idx in range(len(tokens_list)) if idx not in indices_to_remove]
    filtered_activation_list = [activation_list[idx] for idx in range(len(tokens_list)) if idx not in indices_to_remove]

    num_to_pad = original_length - len(filtered_tokens_list)
    if num_to_pad > 0:
        filtered_tokens_list.extend([filtered_tokens_list[-1]] * num_to_pad)
        filtered_activation_list.extend([filtered_activation_list[-1]] * num_to_pad)

    return torch.tensor(filtered_tokens_list), torch.tensor(filtered_activation_list)

In [6]:
t, a = filter_and_pad_tensor(s6c5["tokens"][0], s6c5["activation"][0])
print(len(t.tolist()))

8192


In [7]:
# check if activation align with tokens

print(s6c5["tokens"][0][36])
print(s6c5["tokens"][0].tolist().index(74))
print(s6c5["tokens"][0][5179])
print(s6c5["tokens"][0].tolist().index(128001))

print(t[31])
print(t.tolist().index(74))
print(t[5167])
print(t.tolist().index(128001))

print(s6c5["activation"][0][36])
print(a[31])
print(s6c5["activation"][0][5179])
print(a[5167])

# check

tensor(74)
36
tensor(128001)
5179
tensor(74)
31
tensor(128001)
5167
tensor([-0.0356, -0.0420, -0.1719,  ...,  0.0190, -0.2021,  0.2266],
       dtype=torch.bfloat16)
tensor([-0.0356, -0.0420, -0.1719,  ...,  0.0190, -0.2021,  0.2266])
tensor([ 0.1064, -0.1123, -0.0105,  ..., -0.3047,  0.1582, -0.3359],
       dtype=torch.bfloat16)
tensor([ 0.1064, -0.1123, -0.0105,  ..., -0.3047,  0.1582, -0.3359])


In [8]:
s6c5_i = l(
    "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/reasondata-i-2d-l15/blocks.15.hook_resid_post/shard-6-chunk-00000005.safetensors"
)

In [9]:
t_i, a_i = filter_and_pad_tensor(s6c5_i["tokens"][0], s6c5_i["activation"][0])
print(s6c5_i["tokens"][0].tolist())
print(t_i.tolist())
print(s6c5["tokens"][0].tolist())
print(t.tolist())

[128000, 128006, 882, 128007, 10267, 198, 79145, 69, 2120, 11992, 59, 9594, 2120, 61, 18, 12, 19, 87, 61, 17, 10, 20, 87, 12, 17, 73441, 933, 2746, 584, 1095, 400, 69, 48922, 7, 74, 9317, 3, 79164, 279, 400, 74, 3, 339, 32905, 315, 400, 69, 55976, 12849, 400, 69, 48922, 7, 605, 9317, 7, 16, 8, 13244, 1789, 279, 25491, 315, 420, 3575, 11, 5296, 430, 400, 605, 5947, 18509, 19272, 15, 13244, 128006, 78191, 128007, 128020, 198, 33413, 11, 1095, 596, 1518, 13, 358, 1205, 311, 12849, 279, 220, 605, 339, 32905, 315, 279, 734, 282, 2120, 8, 284, 8119, 2120, 44301, 482, 220, 19, 87, 30556, 489, 220, 20, 87, 482, 220, 17, 8, 26126, 520, 865, 284, 220, 16, 13, 89290, 11, 430, 10578, 17395, 13, 2030, 7344, 1070, 596, 264, 14397, 1618, 1606, 315, 279, 3230, 1486, 865, 284, 220, 16, 13, 6914, 757, 1781, 382, 5451, 11, 1095, 757, 24564, 279, 5811, 4871, 279, 76359, 734, 25, 865, 44301, 482, 220, 19, 87, 30556, 489, 220, 20, 87, 482, 220, 17, 13, 10926, 422, 358, 20206, 304, 865, 284, 220, 16, 11, 430

In [10]:
print(s6c5_i["tokens"][0][34])
print(s6c5_i["tokens"][0].tolist().index(74))
print(s6c5_i["tokens"][0][5175])
print(s6c5_i["tokens"][0].tolist().index(128009))

print(t_i[31])
print(t_i.tolist().index(74))
print(t_i[5167])
print(t_i.tolist().index(128009))

print(s6c5_i["activation"][0][34])
print(a_i[31])
print(s6c5_i["activation"][0][5175])
print(a_i[5167])

tensor(74)
34
tensor(128009)
5175
tensor(74)
31
tensor(128009)
5167
tensor([-0.0312, -0.0258, -0.1494,  ..., -0.0635, -0.0381,  0.1191],
       dtype=torch.bfloat16)
tensor([-0.0312, -0.0258, -0.1494,  ..., -0.0635, -0.0381,  0.1191])
tensor([-0.0203, -0.0269,  0.1167,  ..., -0.0269, -0.2695,  0.1797],
       dtype=torch.bfloat16)
tensor([-0.0203, -0.0269,  0.1167,  ..., -0.0269, -0.2695,  0.1797])


In [11]:
s6c5_o = l(
    "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/reasondata-o-2d-l15/blocks.15.hook_resid_post/shard-6-chunk-00000005.safetensors"
)

In [12]:
t_o, a_o = filter_and_pad_tensor(s6c5_o["tokens"][0], s6c5_o["activation"][0])
print(s6c5_o["tokens"][0].tolist())
print(t_o.tolist())
print(s6c5["tokens"][0].tolist())
print(t.tolist())

[128000, 128011, 10267, 198, 79145, 69, 2120, 11992, 59, 9594, 2120, 61, 18, 12, 19, 87, 61, 17, 10, 20, 87, 12, 17, 73441, 933, 2746, 584, 1095, 400, 69, 48922, 7, 74, 9317, 3, 79164, 279, 400, 74, 3, 339, 32905, 315, 400, 69, 55976, 12849, 400, 69, 48922, 7, 605, 9317, 7, 16, 8, 13244, 1789, 279, 25491, 315, 420, 3575, 11, 5296, 430, 400, 605, 5947, 18509, 19272, 15, 13244, 128012, 128013, 198, 33413, 11, 1095, 596, 1518, 13, 358, 1205, 311, 12849, 279, 220, 605, 339, 32905, 315, 279, 734, 282, 2120, 8, 284, 8119, 2120, 44301, 482, 220, 19, 87, 30556, 489, 220, 20, 87, 482, 220, 17, 8, 26126, 520, 865, 284, 220, 16, 13, 89290, 11, 430, 10578, 17395, 13, 2030, 7344, 1070, 596, 264, 14397, 1618, 1606, 315, 279, 3230, 1486, 865, 284, 220, 16, 13, 6914, 757, 1781, 382, 5451, 11, 1095, 757, 24564, 279, 5811, 4871, 279, 76359, 734, 25, 865, 44301, 482, 220, 19, 87, 30556, 489, 220, 20, 87, 482, 220, 17, 13, 10926, 422, 358, 20206, 304, 865, 284, 220, 16, 11, 430, 15858, 9803, 2574, 13, 691

In [13]:
print(s6c5_o["tokens"][0][32])
print(s6c5_o["tokens"][0].tolist().index(74))
print(s6c5_o["tokens"][0][5171])
print(s6c5_o["tokens"][0].tolist().index(128001))

print(t_o[31])
print(t_o.tolist().index(74))
print(t_o[5167])
print(t_o.tolist().index(128001))

print(s6c5_o["activation"][0][32])
print(a_o[31])
print(s6c5_o["activation"][0][5171])
print(a_o[5167])

tensor(74)
32
tensor(128001)
5171
tensor(74)
31
tensor(128001)
5167
tensor([ 0.0908,  0.0217, -0.1992,  ..., -0.1060, -0.1484,  0.1299],
       dtype=torch.bfloat16)
tensor([ 0.0908,  0.0217, -0.1992,  ..., -0.1060, -0.1484,  0.1299])
tensor([ 0.2617, -0.0498, -0.1074,  ..., -0.2070, -0.1826, -0.4258],
       dtype=torch.bfloat16)
tensor([ 0.2617, -0.0498, -0.1074,  ..., -0.2070, -0.1826, -0.4258])


In [14]:
from transformers import AutoTokenizer

tokenizer_o = AutoTokenizer.from_pretrained(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/DeepSeek-R1-Distill-Llama-8B"
)

In [15]:
print(tokenizer_o.decode(s6c5_o["tokens"][0], skip_special_tokens=False))
print(tokenizer_o.decode(t_o, skip_special_tokens=False))

<｜begin▁of▁sentence｜><｜User｜>Let
\[f(x)=\cos(x^3-4x^2+5x-2).\]
If we let $f^{(k)}$ denote the $k$th derivative of $f$, compute $f^{(10)}(1)$. For the sake of this problem, note that $10!=3628800$.<｜Assistant｜><think>
Okay, let's see. I need to compute the 10th derivative of the function f(x) = cos(x³ - 4x² + 5x - 2) evaluated at x = 1. Hmm, that sounds complicated. But maybe there's a trick here because of the specific point x = 1. Let me think.

First, let me analyze the argument inside the cosine function: x³ - 4x² + 5x - 2. Maybe if I plug in x = 1, that simplifies things. Let me compute that:

1³ - 4*1² + 5*1 - 2 = 1 - 4 + 5 - 2 = 0. Oh, so the argument becomes 0 at x = 1. So f(1) = cos(0) = 1. But I need the 10th derivative. Hmm.

Since the argument of the cosine is a cubic polynomial, and we're evaluating at a root of that polynomial (since it's 0 at x=1), maybe we can use a Taylor series expansion around x = 1. That might make taking derivatives easier, especially high-order one

In [16]:
from transformers import AutoModelForCausalLM

path_b = "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/Llama-3.1-8B"
tokenizer_b = AutoTokenizer.from_pretrained(path_b)
model_b = AutoModelForCausalLM.from_pretrained(path_b)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
print(tokenizer_b.decode(s6c5["tokens"][0], skip_special_tokens=False))
print(tokenizer_b.decode(t, skip_special_tokens=False))

<|begin_of_text|><|q_start|>Q: <|q_end|>Let
\[f(x)=\cos(x^3-4x^2+5x-2).\]
If we let $f^{(k)}$ denote the $k$th derivative of $f$, compute $f^{(10)}(1)$. For the sake of this problem, note that $10!=3628800$.<|a_start|>A: <|a_end|> <think>
Okay, let's see. I need to compute the 10th derivative of the function f(x) = cos(x³ - 4x² + 5x - 2) evaluated at x = 1. Hmm, that sounds complicated. But maybe there's a trick here because of the specific point x = 1. Let me think.

First, let me analyze the argument inside the cosine function: x³ - 4x² + 5x - 2. Maybe if I plug in x = 1, that simplifies things. Let me compute that:

1³ - 4*1² + 5*1 - 2 = 1 - 4 + 5 - 2 = 0. Oh, so the argument becomes 0 at x = 1. So f(1) = cos(0) = 1. But I need the 10th derivative. Hmm.

Since the argument of the cosine is a cubic polynomial, and we're evaluating at a root of that polynomial (since it's 0 at x=1), maybe we can use a Taylor series expansion around x = 1. That might make taking derivatives easier, esp

In [38]:
# context_text = r'''<|begin_of_text|><|q_start|>Q: <|q_end|>Let
# \[f(x)=\cos(x^3-4x^2+5x-2).\]
# If we let $f^{(k)}$ denote the $k$th derivative of $f$, compute $f^{(10)}(1)$. For the sake of this problem, note that $10!=3628800$.<|a_start|>A: <|a_end|> <think>'''
# context_text = "<|begin_of_text|><|q_start|>Q: <|q_end|>Tell me what is a dog.<|a_start|>A: <|a_end|> <think>"


# input_ids = tokenizer_b.encode(context_text, return_tensors="pt").to(device)

# device = 'cuda'
# model_b.to(device)
for i in range(10):
    output_ids = model_b.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=128016,
    )

    # 解码生成文本
    completed_text = tokenizer_b.decode(output_ids[0], skip_special_tokens=False)

    with open("meanemb_special.txt", "a", encoding="utf-8") as file:
        file.write(completed_text + "\n")

In [40]:
# vanilla_base = AutoModelForCausalLM.from_pretrained("/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/models/Llama-3.1-8B")
# vanilla_tokenizer = AutoTokenizer.from_pretrained("/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/models/Llama-3.1-8B")
# vanilla_context_text = "<|begin_of_text|>Q: Tell me what is a dog.A: "


# vanilla_input_ids = vanilla_tokenizer.encode(vanilla_context_text, return_tensors="pt").to('cuda')

# vanilla_base.to('cuda')

for i in range(10):
    vanilla_output_ids = vanilla_base.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=128016,
    )

    # 解码生成文本
    completed_text = vanilla_tokenizer.decode(vanilla_output_ids[0], skip_special_tokens=False)

    with open("zeroemb_special.txt", "a", encoding="utf-8") as file:
        file.write(completed_text + "\n")

In [ ]:
model_b = AutoModelForCausalLM.from_pretrained(path_b, output_attentions=True, attn_implementation="eager")

with torch.no_grad():
    outputs = model_b(input_ids, output_attentions=True)

attentions = outputs.attentions  # (num_layers, batch_size, num_heads, seq_length, seq_length)
tokens = tokenizer_b.convert_ids_to_tokens(input_ids[0])

# 选择一个层和头部进行可视化
layer_idx = -1  # 最后一层
head_idx = 0  # 第一个注意力头

attention = attentions[layer_idx][0, head_idx].cpu().numpy()  # (seq_length, seq_length)

# 绘制 attention 热力图
plt.figure(figsize=(10, 8))
sns.heatmap(attention, xticklabels=tokens, yticklabels=tokens, cmap="viridis", cbar=True)
plt.title(f"Attention Heatmap (Layer {layer_idx+1}, Head {head_idx+1})", fontsize=14)
plt.xlabel("Key Tokens")
plt.ylabel("Query Tokens")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


NameError: name 'plt' is not defined